In [ ]:
 !pip install yfinance
 !pip install yahoofinancials
 !pip install investpy
 !pip install pandas

In [ ]:
#import библиотек
import yfinance as yf
import yahoofinancials
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') # специальное отображение графиков для pyplot

In [ ]:
Gold = yf.download('Gold',
                      start='2015-01-01',
                      end='2023-12-31',
                      progress=False)
Gold

In [ ]:
plt.figure(figsize = (16,8))
plt.title('Закрыть историю цен')
plt.plot(Gold['Close'])
plt.xlabel('Дата')
plt.ylabel('Цена закрытия в долларах США')
plt.show()

In [ ]:
# Создаем новый датафрейм только с колонкой "Close"
data = Gold.filter(['Close'])
# преобразовываем в нумпаевский массив
dataset = data.values
# Вытаскиваем количество строк в дате для обучения модели (LSTM)
training_data_len = math.ceil(len(dataset) * .8)

training_data_len

In [ ]:
#Масштабировать данные (масштабируем)
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

In [ ]:
# Создаем датасет для обучения
train_data = scaled_data[0:training_data_len]
# разбиваем на x underscore train и y underscore train
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i])
    y_train.append(train_data[i])

In [ ]:
#Конвертируем x_train и y_train в нумпаевский массив
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
#Изменить форму данных
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],x_train.shape[2]))

In [ ]:
#Строим нейронку
model = Sequential()
model.add(LSTM(50,return_sequences = True, input_shape = (x_train.shape[1],x_train.shape[2])))
model.add(LSTM(50,return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
#Компилируем модель
model.compile(optimizer='adam',loss = 'mean_squared_error')

In [ ]:
#Тренируем модель
model.fit(x_train,y_train,batch_size = 1, epochs = 50)

In [ ]:
#Создаем тестовый датасет
test_data = scaled_data[training_data_len - 60:]
#по аналогии создаем x_test и y_test
x_test = []
y_test = dataset[training_data_len:]
for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i])

In [ ]:
#опять преобразуем в нумпаевский массив
x_test = np.array(x_test)

In [ ]:
#опять делаем reshape
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],x_test.shape[2]))

In [ ]:
#Получаем модель предсказывающую значения
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
#Получим mean squared error (RMSE) - метод наименьших квадратов
rmse =np.sqrt(np.mean(predictions-y_test)**2)
rmse

In [ ]:
#Строим график
train = data[:training_data_len]
valid = data[training_data_len: ]
valid['Predictions'] = predictions
#Визуализируем
plt.figure(figsize=(16,8))
plt.title('Модель LSTM')
plt.xlabel('Дата', fontsize = 18)
plt.ylabel('Цена закрытия',fontsize = 18)
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])
plt.legend(['Тренироваться','Валь','пред'], loc = 'lower right')
plt.show()